In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import TimeoutException #超时的异常
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import os

from gri_parser import gri_parser
from gri_output import gri_output

In [2]:
gp = gri_parser()
go = gri_output('test.db')

In [3]:
# on X1
driver = webdriver.Chrome()

driver.get("https://database.globalreporting.org")

In [ ]:
select_element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.NAME, "results-datatable_length"))
    )
select_element.click()
time.sleep(1)
Select(select_element).select_by_value('100')
select_element.click()
time.sleep(10)


In [ ]:
while True:
    table = WebDriverWait(driver,10).until(
        EC.presence_of_all_elements_located((By.ID,'results-datatable'))
    )
    content = driver.page_source
    
    data_dict = gp.index_parser(content)
    df1 = go.output_base(data_dict)
    df2 = go.output_reports(data_dict)
    time.sleep(10)
    # 3. 判断next是否可以click是
    next_page = driver.find_element_by_css_selector('#results-datatable_next > a')
    try:
        next_page.click()
        time.sleep(200)
    except:
        break

In [ ]:
next_page = driver.find_element_by_css_selector('#results-datatable_next > a')
next_page.click()

In [ ]:
df1 = go.output_base(data_dict)

In [ ]:
driver.quit()

In [ ]:
for k in data_dict.keys():
    print(k)

# 1. downloader

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')#无头模式，服务器没有图形界面这个必须
chrome_options.add_argument('--disable-gpu')#不需要gpu加速
chrome_options.add_argument('--no-sandbox') # 这个配置很重要
chrome_options.add_argument("user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'")
driver = webdriver.Chrome(chrome_options=chrome_options)    # 如果没有把chromedriver加入到PATH中，就需要指明路径
gp = gri_parser()
go = gri_output('../test.db')
driver.get("https://database.globalreporting.org/search/")

# 1. 等待加载之后处理复选框
try:
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "results-datatable_length"))
    )
    Select(select_element).select_by_value('100')
finally:
    driver.quit()
# 2. 等待table加载完成后保存网页
try:
    table = WebDriverWait(driver,10).until(
        EC.presence_of_all_elements_located((By.ID,'results-datatable'))
    )
    content = driver.page_source
    data_dict = gp.index_parser(content)
    df1 = go.output_base(data_dict)
    df2 = go.output_reports(data_dict)
    time.sleep(10)
# 3. 判断next是否可以click是
    next_page = driver.find_element_by_css_selector('#results-datatable_wrapper > div:nth-child(3) > div.col-sm-7 > div > ul > li.paginate_button.page-item.next')
    if next_page.is_enable():
        next_page.click()
finally:
    driver.quit()


# 2. parser

In [ ]:
%%writefile gri_parser.py
from bs4 import BeautifulSoup
class gri_parser():
    '''
    parser the search result of https://database.globalreporting.org/search/
    
    '''
    def index_parser(self,content):
        '''
        20201022
        to parser the search result
        Args:
            content (binary or str): the html source of page
        Returns:
            dict: a data dictionary
        '''
        # data container
        data_dict = {'ORG_name':[],'ORG_url':[],'size':[],'sector':[],'country':[], 'region':[], 'reports':[]}
        
        # parse
        soup = BeautifulSoup(content, 'html.parser')
        table = soup.find('table',{'class':'table table-hover dataTable'})
        tablebody = table.tbody
        for tr in tablebody.find_all('tr'):
            tmp_dict = {}
            tds = tr.find_all('td')
            tmp_dict['ORG_name'] = tds[0].get_text().strip()
            tmp_dict['ORG_url'] = tds[0].h4.a['href']
            tmp_dict['size'] = tds[1].get_text()
            tmp_dict['sector'] = tds[2].get_text()
            tmp_dict['country'] = tds[3].img['src']
            tmp_dict['region'] = tds[4].get_text()
            tmp_dict['reports'] = []
            
            for a in tds[5].find_all('a'):
                tmp_report = {}
                tmp_report['report_url'] = a['href']
                tmp_report['report_year'] = a.span.contents[0].replace('-','').strip()
                tmp_report['report_label'] = a.find('span',{'class':'text-slim label label-primary'}).get_text().strip()
                tmp_dict['reports'].append(tmp_report)
            
            for k in data_dict.keys():
                data_dict[k].append(tmp_dict.get(k))
        return data_dict

# if __name__ == '__main__':
with open('database.globalreporting.org_search.html','rb') as f:
    content = f.read()
gp = gri_parser()
data_dict = gp.index_parser(content)
# print(data_dict)

# 3. output

In [ ]:
%%writefile gri_output.py
import pandas as pd
import sqlite3

class gri_output():
    '''
    for data save
    Attributes:
        dbpath (str): the database path
    '''
    def __init__(self, dbpath=None):
        '''
        Args:
            dbpath (str): the database path, default to None
        '''
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath)
    def output_base(self,data_dict,table_name='gri_base'):
        '''
        for save page to database
        Args:
            data_dict (dict): the data
            table_name (str): the table name to store the data
        '''
        df = pd.DataFrame(data_dict)
        df = df[['ORG_name','ORG_url','size','sector','country', 'region']]
        df.to_sql(name=table_name,con=self.conn,if_exists='append',index=False)
        return df
    def output_reports(self,data_dict,table_name='gri_reports'):
        tmp_dict = {'ORG_url':[],'report_url':[],'report_year':[],'report_label':[]}
        for u,rs in zip(data_dict['ORG_url'],data_dict['reports']):
            if not rs:
                continue
            for r in rs:
                tmp_dict['ORG_url'].append(u)
                for k in list(tmp_dict.keys())[1:]:
                    tmp_dict[k].append(r.get(k))
        df = pd.DataFrame(tmp_dict)
        df.to_sql(name=table_name,con=self.conn,if_exists='append',index=False)
        return df

# 处理flag

In [2]:
import requests

In [4]:
r = requests.get('https://database.globalreporting.org/organizations/8738/',verify=False)

D:\software\Anaconda\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'database.globalreporting.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
r.status_code

200

In [8]:
with open('page.html','ab+') as f:
    f.write(r.content)